In [1]:
import torch
from torch.utils.data import Dataset, random_split
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os.path
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
dev = 'mps' if torch.has_mps else 'cpu'
print(dev)

mps


In [3]:
device = torch.device(dev)

model = nn.Sequential(
    nn.Linear(539, 2156),
    nn.BatchNorm1d(2156),
    nn.ReLU(),
    nn.Linear(2156, 2156),
    nn.BatchNorm1d(2156),
    nn.ReLU(),
    nn.Linear(2156, 2156),
    nn.BatchNorm1d(2156),
    nn.ReLU(),
    nn.Linear(2156, 2156),
    nn.BatchNorm1d(2156),
    nn.ReLU(),
    nn.Linear(2156, 2156),
    nn.BatchNorm1d(2156),
    nn.ReLU(),
    nn.Linear(2156, 2156),
    nn.BatchNorm1d(2156),
    nn.ReLU(),
    nn.Linear(2156, 1)
)

model.load_state_dict(torch.load('./jun7_930am.pth'))
model.to(device)
model.eval()

Sequential(
  (0): Linear(in_features=539, out_features=2156, bias=True)
  (1): BatchNorm1d(2156, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=2156, out_features=2156, bias=True)
  (4): BatchNorm1d(2156, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Linear(in_features=2156, out_features=2156, bias=True)
  (7): BatchNorm1d(2156, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Linear(in_features=2156, out_features=2156, bias=True)
  (10): BatchNorm1d(2156, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Linear(in_features=2156, out_features=2156, bias=True)
  (13): BatchNorm1d(2156, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU()
  (15): Linear(in_features=2156, out_features=2156, bias=True)
  (16): BatchNorm1d(2156, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (17)

In [4]:
test_data = pd.read_csv('./csv_artifacts/test_dataish.csv', dtype=np.float32)
trip_ids = pd.read_csv('./csv_artifacts/test_public.csv')
trip_ids = trip_ids[['TRIP_ID']]
all_xy = test_data.to_numpy()
test_input = torch.tensor(all_xy[:,0:539], dtype=torch.float32).to(device)

In [5]:
with torch.no_grad():
    outputs = model(test_input)

In [6]:
outputs = torch.Tensor.cpu(outputs)

In [7]:
output_file = pd.DataFrame(columns=['TRIP_ID', 'TRAVEL_TIME'])
display(output_file)

,TRIP_ID,TRAVEL_TIME


In [8]:
for count, i in enumerate(outputs):
    trip_id = trip_ids.iloc[count].to_string()
    trip_id = trip_id.replace('TRIP_ID', '').strip()
    print(trip_id)
    item = i.item()
    print('Trip ID: {} Result: {}'.format(count+1, item))
    output_file.loc[len(output_file)] = [trip_id, item]

T1
Trip ID: 1 Result: 852.994384765625
T2
Trip ID: 2 Result: 846.5121459960938
T3
Trip ID: 3 Result: 794.086669921875
T4
Trip ID: 4 Result: 689.9039306640625
T5
Trip ID: 5 Result: 688.7111206054688
T6
Trip ID: 6 Result: 1012.1104125976562
T7
Trip ID: 7 Result: 864.951171875
T8
Trip ID: 8 Result: 1036.5166015625
T9
Trip ID: 9 Result: 742.4293212890625
T10
Trip ID: 10 Result: 845.3755493164062
T11
Trip ID: 11 Result: 677.4452514648438
T12
Trip ID: 12 Result: 1019.314208984375
T13
Trip ID: 13 Result: 455.5355224609375
T14
Trip ID: 14 Result: 786.1820678710938
T15
Trip ID: 15 Result: 824.8909301757812
T16
Trip ID: 16 Result: 716.4857177734375
T17
Trip ID: 17 Result: 728.7537231445312
T18
Trip ID: 18 Result: 776.0363159179688
T19
Trip ID: 19 Result: 794.2728881835938
T20
Trip ID: 20 Result: 995.4447021484375
T21
Trip ID: 21 Result: 748.0079956054688
T22
Trip ID: 22 Result: 1001.1544799804688
T23
Trip ID: 23 Result: 746.4452514648438
T24
Trip ID: 24 Result: 1060.17578125
T25
Trip ID: 25 Resu

In [9]:
output_file.to_csv('submissions_2.csv', index=False)